In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Calculate time horizon for loss of potential predictability

In [50]:
import xarray as xr
import numpy as np
import gfdl_utils as gu
import glob
from dask.diagnostics import ProgressBar
from information import *
from processing import *
from variance import *
import os

In [52]:
ppname = 'ocean_month'
frequency = gu.core.get_timefrequency(ppcontrol,ppname)
ppp = xr.open_zarr(ppeDict['datasavedir']+'/processed/ppp/'+ppname+'.zarr')

In [48]:
def calc_timehorizon(da,threshold,dim='time',smooth=None,):
    ''' Find time at which value of [da] drops below [threshold], moving forward in time.
    If it never drops below threshold, np.max(time)+1 is returned.'''
    if smooth is not None:
        tmp = da.rolling({dim:smooth},center=True,min_periods=1).mean()
    else:
        tmp = da.copy()
    
    return xr.where(tmp<threshold, tmp[dim],np.max(da[dim])+1).min(dim=dim,skipna=True)

In [49]:
threshold = 0.235559205
smooth = 12

savedir = ppeDict['datasavedir']+'/processed/ppp_timehorizon/'

for variable,da in ppp.items():
    savefilename = '.'.join([variable,frequency,'smooth_'+str(smooth),'nc'])
    if os.path.isfile(savedir+savefilename):
            print(savefilename+' already exists.')
            continue
    else:
        tmp = calc_timehorizon(da,threshold,dim='time',smooth=smooth)
        tmp.name = variable
        print('Saving '+savefilename)
        with ProgressBar():
            tmp.to_netcdf(savedir+savefilename)

Saving intpp.monthly.smooth_12.nc
[########################################] | 100% Completed | 3.21 sms
Saving intppdiat.monthly.smooth_12.nc
[########################################] | 100% Completed | 3.29 sms
Saving intppdiaz.monthly.smooth_12.nc
[########################################] | 100% Completed | 3.30 sms
Saving intppmisc.monthly.smooth_12.nc
[########################################] | 100% Completed | 3.18 sms
Saving intppnitrate.monthly.smooth_12.nc
[########################################] | 100% Completed | 3.20 sms
Saving intpppico.monthly.smooth_12.nc
[########################################] | 100% Completed | 3.30 sms
